# Correct some typos in the scan names, and make sure that the files have attributes listing what the scans show.

This was run interactively to make sure all scan names are consistent

In [1]:
from pathlib import Path
import patato as pat
import re

In [2]:
data = Path("../")

In [3]:
allowable_regions = ["bicep", "forearm", "leg", "neck"]
allowable_regions += ["brachialartery", "upperforearm", "ulnar"]

pattern_regions = "|".join([f"(?:{x})" for x in allowable_regions])

position_descriptors = [
    "parallel",
    "pigment",
    "nopigment",
    "paralleltwopigment",
    "parallelalt",
]
pattern_position = "|".join([f"(?:{x})" for x in position_descriptors])

allowable_scan_name_pattern = (
    r"^(?:((?:left)|(?:right))-)?("
    + pattern_regions
    + ")(?:-("
    + pattern_position
    + "))?(?:([0-9])|-)([a-z])$"
)
close_scan_name_pattern = (
    r"^(?:((?:left)|(?:right))-)?("
    + pattern_regions
    + ")(?:-("
    + pattern_position
    + "))?-(?:([0-9])|-)([a-z])$"
)

ignore_scan_name = "Scan_[0-9]*"

print(allowable_scan_name_pattern)

^(?:((?:left)|(?:right))-)?((?:bicep)|(?:forearm)|(?:leg)|(?:neck)|(?:brachialartery)|(?:upperforearm)|(?:ulnar))(?:-((?:parallel)|(?:pigment)|(?:nopigment)|(?:paralleltwopigment)|(?:parallelalt)))?(?:([0-9])|-)([a-z])$


In [4]:
print(re.findall(allowable_scan_name_pattern, "forearm-paralleltwopigment-a"))
print(re.match(ignore_scan_name, "Scan_50"))

[('', 'forearm', 'paralleltwopigment', '', 'a')]
<re.Match object; span=(0, 7), match='Scan_50'>


In [5]:
s = set()

for f in data.glob("**/*.hdf5"):
    p = pat.PAData.from_hdf5(f, "r")
    if re.findall(allowable_scan_name_pattern, p.get_scan_name()):
        continue

    # Interactively apply correction to scan names here:

    s.add(p.get_scan_name())
    p.scan_reader.file.close()